# Stereo-vision triangulation

Notes on stereo-vision triangulation. We discuss different strategies to perform structure estimation with a stereo-vision system. In order to do that, we will use the following dataset of undistorted correspondences of points, and internal and external parameters of the two views.

In [1]:
import numpy as np

# Correspondences of points
x1 = np.array([825.8985226149575, 335.48621768716475]) # first view
x2 = np.array([606.8071528366432, 361.80915742993350]) # second view

# Internal matrices
K1 = np.array([[2619.30346254742, 3.7714090338749293, 657.957197097675],
               [0.0, 2617.624139233695, 564.4749343876285],
               [0.0, 0.0, 1.0]])

K2 = np.array([[2520.489284681429, 1.698115677245312, 626.9344804237346],
               [0.0, 2520.190708188707, 545.3142600064642],
               [0.0, 0.0, 1.0]])

# External parameters: position and orientation of first view relative to the second
R = np.array([[0.925942148377286, -0.0036025297155779026, 0.3776481955975754],
              [-0.002941633180521065, 0.9998553735468453, 0.01675048607443406],
              [-0.3776539218023393, -0.016620883524859024, 0.9257976353275945]])

t = np.array([-375.33478798894356, -1.6124027666068885, 82.54764531599906])


# Projection matrices
P1 = K1 @ np.c_[np.eye(3), np.zeros(3)]
P2 = K2 @ np.c_[R, t]

## Linear homogenenous method (DLT):

The homogenenous triangulation method or DLT, is a linear method where we solve for a homogeneous point $\mathbf{X} = [X_1, X_2, X_3, X_4]^\mathsf{T}$. Based on the above, the projection of this point $\mathbf{X}$ in the first view

\begin{equation}
\label{eq:proj1}
s_1 \mathbf{x}_1 = \mathbf{P}_1 \mathbf{X} \enspace,
\end{equation}

and in the second view

\begin{equation}
\label{eq:proj2}
s_2 \mathbf{x}_2 = \mathbf{P}_2 \mathbf{X} \enspace,
\end{equation}


can be written as:


\begin{align}
\label{eq:pinhole1dlt}
s_1 \begin{bmatrix} x_1\\y_1\\1 \end{bmatrix} &  = 
\begin{bmatrix} 
p^{11}_1 & p^{12}_1 & p^{13}_1 & p^{14}_1\\
p^{21}_1 & p^{22}_1 & p^{23}_1 & p^{24}_1\\
p^{31}_1 & p^{32}_1 & p^{33}_1 & p^{34}_1
\end{bmatrix} \begin{bmatrix} X_1\\X_2\\X_3\\X_4 \end{bmatrix} \enspace, \\
\label{eq:pinhole2dlt}
s_2 \begin{bmatrix} x_2\\y_2\\1 \end{bmatrix} &  = 
\begin{bmatrix} 
p^{11}_2 & p^{12}_2 & p^{13}_2 & p^{14}_2\\
p^{21}_2 & p^{22}_2 & p^{23}_2 & p^{24}_2\\
p^{31}_2 & p^{32}_2 & p^{33}_2 & p^{34}_2
\end{bmatrix} \begin{bmatrix} X_1\\X_2\\X_3\\X_4 \end{bmatrix} \enspace.
\end{align}

From Eq.\ref{eq:pinhole1dlt} we can get three equations:

\begin{align}
\label{eq:X1dlt}
s_1 x_1 & = p^{11}_1 X_1 + p^{12}_1 X_2 + p^{13}_1 X_3 + p^{14}_1 X_4 \enspace,\\
\label{eq:Y1dlt}
s_1 y_1 & = p^{21}_1 X_1 + p^{22}_1 X_2 + p^{23}_1 X_3 + p^{24}_1 X_4 \enspace,\\
\label{eq:Z1dlt}
s_1     & = p^{31}_1 X_1 + p^{32}_1 X_2 + p^{33}_1 X_3 + p^{34}_1 X_4 \enspace,\\
\end{align}

Replacing $s_1$ from Eq.\ref{eq:Z1dlt} into Eq.\ref{eq:X1dlt} and Eq.\ref{eq:Y1dlt} we have:

\begin{align}
(p^{31}_1 X_1 + p^{32}_1 X_2 + p^{33}_1 X_3 + p^{34}_1 X_4) x_1 & = p^{11}_1 X_1 + p^{12}_1 X_2 + p^{13}_1 X_3 + p^{14}_1 X_4 \enspace,\\
(p^{31}_1 X_1 + p^{32}_1 X_2 + p^{33}_1 X_3 + p^{34}_1 X_4) y_1 & = p^{21}_1 X_1 + p^{22}_1 X_2 + p^{23}_1 X_3 + p^{24}_1 X_4 \enspace,\\
\end{align}

Grouping $X_1$, $X_2$, $X_3$ and $X_4$ in the above two equations we finally get:

\begin{align}
\label{eq:x1dlt}
(p^{31}_1x_1-p^{11}_1)X_1 + (p^{32}_1x_1-p^{12}_1)X_2 + (p^{33}_1x_1-p^{13}_1)X_3 + (p^{34}_1x_1-p^{14}_1)X_4 & = 0 \enspace,\\
\label{eq:y1dlt}
(p^{31}_1y_1-p^{21}_1)X_1 + (p^{32}_1y_1-p^{22}_1)X_2 + (p^{33}_1y_1-p^{23}_1)X_3 + (p^{34}_1y_1-p^{24}_1)X_4 & = 0 \enspace.\\
\end{align}



In the same way, from from Eq.\ref{eq:pinhole2dlt} we get three equations:

\begin{align}
\label{eq:X2dlt}
s_2 x_2 & = p^{11}_2 X_1 + p^{12}_2 X_2 + p^{13}_2 X_3 + p^{14}_2 X_4 \enspace,\\
\label{eq:Y2dlt}
s_2 y_2 & = p^{21}_2 X_1 + p^{22}_2 X_2 + p^{23}_2 X_3 + p^{24}_2 X_4 \enspace,\\
\label{eq:Z2dlt}
s_2     & = p^{31}_2 X_1 + p^{32}_2 X_2 + p^{33}_2 X_3 + p^{34}_2 X_4 \enspace,\\
\end{align}

which, by replacing $s_2$ from Eq.\ref{eq:Z2dlt} into Eq.\ref{eq:X2dlt} and Eq.\ref{eq:Y2dlt} and grouping the unknowns $X_1$, $X_2$, $X_3$ and $X_4$, we also obtain two equations:
\begin{align}
\label{eq:x2dlt}
(p^{31}_2x_2-p^{11}_2)X_1 + (p^{32}_2x_2-p^{12}_2)X_2 + (p^{33}_2x_2-p^{13}_2)X_3 + (p^{34}_2x_2-p^{14}_2)X_4 & = 0\enspace,\\
\label{eq:y2dlt}
(p^{31}_2y_2-p^{21}_2)X_1 + (p^{32}_2y_2-p^{22}_2)X_2 + (p^{33}_2y_2-p^{23}_2)X_3 + (p^{34}_2y_2-p^{24}_2)X_4 & = 0\enspace.\\
\end{align}

With equations \ref{eq:x1dlt}, \ref{eq:y1dlt}, \ref{eq:x2dlt}, \ref{eq:y2dlt}, we can express our system matrically as

\begin{equation}
\begin{bmatrix}
p^{31}_1x_1-p^{11}_1 & p^{32}_1x_1-p^{12}_1 & p^{33}_1x_1-p^{13}_1 & p^{34}_1x_1-p^{14}_1 \\
p^{31}_1y_1-p^{21}_1 & p^{32}_1y_1-p^{22}_1 & p^{33}_1y_1-p^{23}_1 & p^{34}_1y_1-p^{24}_1 \\
p^{31}_2x_2-p^{11}_2 & p^{32}_2x_2-p^{12}_2 & p^{33}_2x_2-p^{13}_2 & p^{34}_2x_2-p^{14}_2 \\
p^{31}_2y_2-p^{21}_2 & p^{32}_2y_2-p^{22}_2 & p^{33}_2y_2-p^{23}_2 & p^{34}_2y_2-p^{24}_2
\end{bmatrix} 
\begin{bmatrix}
X_1\\X_2\\X_3\\X_4
\end{bmatrix} = 
\begin{bmatrix}
0\\0\\0\\0
\end{bmatrix} \enspace,
\end{equation}

where we have a system with four equations and a total of four homogeneous unknowns. This is a set of redundant equations due to the solution is determined only up to scale.

We can write this system more compactly as

\begin{equation}
\label{eq:DLT1}
\begin{bmatrix}
x_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{1\mathsf{T}}\\
y_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{2\mathsf{T}}\\
x_2 \mathbf{P}_2^{3\mathsf{T}} - \mathbf{P}_2^{1\mathsf{T}}\\
y_2 \mathbf{P}_2^{3\mathsf{T}} - \mathbf{P}_2^{2\mathsf{T}}
\end{bmatrix} 
\begin{bmatrix}
X_1\\X_2\\X_3\\X_4
\end{bmatrix} = 
\begin{bmatrix}
0\\0\\0\\0
\end{bmatrix} \enspace,
\end{equation}

where $\mathbf{P}_i^j$ represent the $j$-th row of the $i$-th projection matrix as column vector.

An implementation of this triangulation method using our dataset can be the following

In [2]:
A = np.array([x1[0]*P1[2] - P1[0],
              x1[1]*P1[2] - P1[1],
              x2[0]*P2[2] - P2[0],
              x2[1]*P2[2] - P2[1]])

# Solve the sistem Ax = 0
u, s, vh = np.linalg.svd(A)

X1 = vh[-1]
X1 = X1[:3]/X1[-1]
print('Method 1 (homogeneous solution DLT):\n{}'.format(X1))

Method 1 (homogeneous solution DLT):
[ 54.13825004 -73.74546967 842.70532166]


The OpenCV function `triangulatePoints` implements the DLT method. You can check it in the [source code](https://github.com/opencv/opencv/blob/master/modules/calib3d/src/triangulate.cpp#L54). We will use the function to compare the results with `X1` obtained with the above code. The documentation of the function is [here](https://docs.opencv.org/4.2.0/d9/d0c/group__calib3d.html#gad3fc9a0c82b08df034234979960b778c).

In [3]:
import cv2
X2 = cv2.triangulatePoints(P1, P2, x1, x2)
X2 = (X2[:3]/X2[-1]).flatten()
print('Method 2 (OpenCV function):\n{}'.format(X2))

Method 2 (OpenCV function):
[ 54.13825004 -73.74546967 842.70532166]


As you can check, the results are the same. It is better use the OpenCV function in case you have more than one point to triangulate because the only way to do that is iteratively for each point and `triangulatePoints` is optimized and is written in C++ which make it more fast and efficient.

### DLT method using cross product

We can address the DLT strategy by using the cross product in Eq.\ref{eq:proj1} and in Eq.\ref{eq:proj2}, in order to "remove" the scale factors $s_1$ and $s_2$. That is

\begin{align}
\label{eq:cross1}
\mathbf{x}_1 \times (\mathbf{P}_1 \mathbf{X}) &  = 0 \enspace, \\
\label{eq:cross2}
\mathbf{x}_2 \times (\mathbf{P}_2 \mathbf{X}) &  = 0  \enspace.
\end{align}

We can write Eq.\ref{eq:cross1} as

\begin{equation}
\begin{bmatrix}
x_1\\ y_1\\ 1
\end{bmatrix} \times
\begin{bmatrix}
\mathbf{P}_1^{1\mathsf{T}} \mathbf{X}\\
\mathbf{P}_1^{2\mathsf{T}} \mathbf{X}\\
\mathbf{P}_1^{3\mathsf{T}} \mathbf{X}
\end{bmatrix} = 
\begin{bmatrix}
0\\0\\0
\end{bmatrix} \enspace,
\end{equation}

where $\mathbf{P}_1^{i\mathsf{T}}$ represents the $i$-th row of the projection matrix $\mathbf{P}_1$ of the first view. Performing the cross product we have:

\begin{equation}
\begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
x_1 &  y_1 & 1 \\
\mathbf{P}_1^{1\mathsf{T}} \mathbf{X} & \mathbf{P}_1^{2\mathsf{T}} \mathbf{X} & \mathbf{P}_1^{3\mathsf{T}} \mathbf{X}
\end{vmatrix} = 
\begin{bmatrix}
0\\0\\0
\end{bmatrix} 
\;\; \longrightarrow \;\;  
\begin{bmatrix}
y_1 \mathbf{P}_1^{3\mathsf{T}} \mathbf{X} - \mathbf{P}_1^{2\mathsf{T}} \mathbf{X} \\
x_1 \mathbf{P}_1^{3\mathsf{T}} \mathbf{X} - \mathbf{P}_1^{1\mathsf{T}} \mathbf{X} \\
x_1 \mathbf{P}_1^{2\mathsf{T}} \mathbf{X} - y_1 \mathbf{P}_1^{1\mathsf{T}} \mathbf{X}
\end{bmatrix} = 
\begin{bmatrix}
0\\0\\0
\end{bmatrix} 
\;\; \longrightarrow \;\;
\begin{bmatrix}
y_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{2\mathsf{T}} \\
x_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{1\mathsf{T}} \\
x_1 \mathbf{P}_1^{2\mathsf{T}} - y_1 \mathbf{P}_1^{1\mathsf{T}}
\end{bmatrix} \mathbf{X} = 
\begin{bmatrix}
0\\0\\0
\end{bmatrix} 
\enspace.
\end{equation}

From the above three equantions only the first two are linearly independet. This is why we remove the last equation from the system, and in this way we get:

\begin{equation}
\label{eq:cross_system1}
\begin{bmatrix}
x_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{1\mathsf{T}} \\
y_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{2\mathsf{T}}
\end{bmatrix} \mathbf{X} = 
\begin{bmatrix}
0\\0
\end{bmatrix} 
\enspace.
\end{equation}


Now, from the cross product in Eq.\ref{eq:cross2} we also get:

\begin{equation}
\label{eq:cross_system2}
\begin{bmatrix}
x_2 \mathbf{P}_2^{3\mathsf{T}} - \mathbf{P}_2^{1\mathsf{T}} \\
y_2 \mathbf{P}_2^{3\mathsf{T}} - \mathbf{P}_2^{2\mathsf{T}}
\end{bmatrix} \mathbf{X} = 
\begin{bmatrix}
0\\0
\end{bmatrix} 
\enspace.
\end{equation}

Merging Eq.\ref{eq:cross_system1} and Eq.\ref{eq:cross_system2}, we get the system


\begin{equation}
\begin{bmatrix}
x_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{1\mathsf{T}}\\
y_1 \mathbf{P}_1^{3\mathsf{T}} - \mathbf{P}_1^{2\mathsf{T}}\\
x_2 \mathbf{P}_2^{3\mathsf{T}} - \mathbf{P}_2^{1\mathsf{T}}\\
y_2 \mathbf{P}_2^{3\mathsf{T}} - \mathbf{P}_2^{2\mathsf{T}}
\end{bmatrix} \mathbf{X} = 
\begin{bmatrix}
0\\0\\0\\0
\end{bmatrix} \enspace,
\end{equation}

which is the same in Eq.\ref{eq:DLT1} stated with a different approach.

### DLT with bilinear relations

Another way to address the DLT method is by expressing Eq.\ref{eq:proj1} and Eq.\ref{eq:proj2} as one equation based on $\mathbf{P}_1 \mathbf{X} - s_1\mathbf{x}_1 = \mathbf{0}$ and $\mathbf{P}_2 \mathbf{X} - s_2\mathbf{x}_2 = \mathbf{0}$

\begin{equation}
\label{eq:bilinear}
\begin{bmatrix}
\mathbf{P}_1 & \mathbf{x}_1 & \mathbf{0}\\
\mathbf{P}_2 & \mathbf{0} & \mathbf{x}_2
\end{bmatrix} \begin{bmatrix}
\mathbf{X} \\ -s_1 \\ -s_2
\end{bmatrix}  = \mathbf{0} \enspace,
\end{equation}

where this equation is used to state the bilinear relations (Part IV, pag. 411, H&Z). Eq.\ref{eq:bilinear} is equivalent to the system

\begin{equation}
\begin{bmatrix}
p^{11}_1 & p^{12}_1 & p^{13}_1 & p^{14}_1 & x_1 & 0\\
p^{21}_1 & p^{22}_1 & p^{23}_1 & p^{24}_1 & y_1 & 0\\
p^{31}_1 & p^{32}_1 & p^{33}_1 & p^{34}_1 & 1 & 0 \\
p^{11}_2 & p^{12}_2 & p^{13}_2 & p^{14}_2 & 0 & x_2\\
p^{21}_2 & p^{22}_2 & p^{23}_2 & p^{24}_2 & 0 & y_2\\
p^{31}_2 & p^{32}_2 & p^{33}_2 & p^{34}_2 & 0 & 1
\end{bmatrix} \begin{bmatrix}
X_1 \\ X_2 \\ X_3 \\ X_4 \\ -s_1 \\ -s_2
\end{bmatrix}  = 
\begin{bmatrix}
0\\0\\0\\0\\0\\0
\end{bmatrix} \enspace,
\end{equation}

that is a $6 \times 6$ set of equations.

In [4]:
a1 = np.c_[P1, np.r_[x1,1], [0,0,0]]
a2 = np.c_[P2, [0,0,0], np.r_[x2,1]]
A = np.r_[a1, a2]

*_, vh = np.linalg.svd(A)
Xn = vh[-1,:4]
Xn = Xn[:3]/Xn[-1]

print('Alternative homogeneous solution:\n{}'.format(Xn))

Alternative homogeneous solution:
[ 54.09581903 -73.7634758  842.94223376]


## Linear inhomogenenous method:

In this method we solve for the inhomogeneous point $(X, Y, Z)$. The projection equantions Eq.\ref{eq:proj1} and Eq.\ref{eq:proj2} of a point $\mathbf{X} = [X, Y, Z, 1]^\mathsf{T}$ given in the world frame can be written as:

\begin{align}
\label{eq:pinhole1}
s_1 \begin{bmatrix} x_1\\y_1\\1 \end{bmatrix} &  = 
\begin{bmatrix} 
p^{11}_1 & p^{12}_1 & p^{13}_1 & p^{14}_1\\
p^{21}_1 & p^{22}_1 & p^{23}_1 & p^{24}_1\\
p^{31}_1 & p^{32}_1 & p^{33}_1 & p^{34}_1
\end{bmatrix} \begin{bmatrix} X\\Y\\Z\\1 \end{bmatrix} \enspace, \\
\label{eq:pinhole2}
s_2 \begin{bmatrix} x_2\\y_2\\1 \end{bmatrix} &  = 
\begin{bmatrix} 
p^{11}_2 & p^{12}_2 & p^{13}_2 & p^{14}_2\\
p^{21}_2 & p^{22}_2 & p^{23}_2 & p^{24}_2\\
p^{31}_2 & p^{32}_2 & p^{33}_2 & p^{34}_2
\end{bmatrix} \begin{bmatrix} X\\Y\\Z\\1 \end{bmatrix} \enspace.
\end{align}

From Eq.\ref{eq:pinhole1} we can get three equations:

\begin{align}
\label{eq:X1}
s_1 x_1 & = p^{11}_1 X + p^{12}_1 Y + p^{13}_1 Z + p^{14}_1 \enspace,\\
\label{eq:Y1}
s_1 y_1 & = p^{21}_1 X + p^{22}_1 Y + p^{23}_1 Z + p^{24}_1 \enspace,\\
\label{eq:Z1}
s_1     & = p^{31}_1 X + p^{32}_1 Y + p^{33}_1 Z + p^{34}_1 \enspace.
\end{align}

Replacing Eq.\ref{eq:Z1} into Eq.\ref{eq:X1} and Eq.\ref{eq:Y1} we have:

\begin{align}
(p^{31}_1 X + p^{32}_1 Y + p^{33}_1 Z + p^{34}_1) x_1 & = p^{11}_1 X + p^{12}_1 Y + p^{13}_1 Z + p^{14}_1 \enspace,\\
(p^{31}_1 X + p^{32}_1 Y + p^{33}_1 Z + p^{34}_1) y_1 & = p^{21}_1 X + p^{22}_1 Y + p^{23}_1 Z + p^{24}_1 \enspace.
\end{align}

Grouping $X$, $Y$ and $Z$ in the above two equations:

\begin{align}
\label{eq:x1}
(p^{31}_1x_1-p^{11}_1)X + (p^{32}_1x_1-p^{12}_1)Y + (p^{33}_1x_1-p^{13}_1)Z & = p^{14}_1-p^{34}_1x_1 \enspace,\\
\label{eq:y1}
(p^{31}_1y_1-p^{21}_1)X + (p^{32}_1y_1-p^{22}_1)Y + (p^{33}_1y_1-p^{23}_1)Z & = p^{24}_1-p^{34}_1y_1 \enspace.
\end{align}



In the same way, from Eq.\ref{eq:pinhole2} we get three equations:

\begin{align}
\label{eq:X2}
s_2 x_2 & = p^{11}_2 X + p^{12}_2 Y + p^{13}_2 Z + p^{14}_2 \enspace,\\
\label{eq:Y2}
s_2 y_2 & = p^{21}_2 X + p^{22}_2 Y + p^{23}_2 Z + p^{24}_2 \enspace,\\
\label{eq:Z2}
s_2     & = p^{31}_2 X + p^{32}_2 Y + p^{33}_2 Z + p^{34}_2 \enspace.
\end{align}

Replacing Eq.\ref{eq:Z2} into Eq.\ref{eq:X2} and Eq.\ref{eq:Y2} and grouping the unknowns $X$, $Y$ and $Z$:
\begin{align}
\label{eq:x2}
(p^{31}_2x_2-p^{11}_2)X + (p^{32}_2x_2-p^{12}_2)Y + (p^{33}_2x_2-p^{13}_2)Z & = p^{14}_2-p^{34}_2x_2 \enspace,\\
\label{eq:y2}
(p^{31}_2y_2-p^{21}_2)X + (p^{32}_2y_2-p^{22}_2)Y + (p^{33}_2y_2-p^{23}_2)Z & = p^{24}_2-p^{34}_2y_2 \enspace.
\end{align}

With equations \ref{eq:x1}, \ref{eq:y1}, \ref{eq:x2}, \ref{eq:y2}, we can express our system matrically as

\begin{equation}
\label{eq:inh}
\begin{bmatrix}
p^{31}_1x_1-p^{11}_1 & p^{32}_1x_1-p^{12}_1 & p^{33}_1x_1-p^{13}_1\\
p^{31}_1y_1-p^{21}_1 & p^{32}_1y_1-p^{22}_1 & p^{33}_1y_1-p^{23}_1\\
p^{31}_2x_2-p^{11}_2 & p^{32}_2x_2-p^{12}_2 & p^{33}_2x_2-p^{13}_2\\
p^{31}_2y_2-p^{21}_2 & p^{32}_2y_2-p^{22}_2 & p^{33}_2y_2-p^{23}_2
\end{bmatrix} 
\begin{bmatrix}
X\\Y\\Z
\end{bmatrix} = 
\begin{bmatrix}
p^{14}_1-p^{34}_1x_1\\
p^{24}_1-p^{34}_1y_1\\
p^{14}_2-p^{34}_2x_2\\
p^{24}_2-p^{34}_2y_2
\end{bmatrix} \enspace,
\end{equation}

where, unlike the DLT method, here we have a system of four equations and three inhomogeneous unknowns, and due to this the 3D point is the least square solution to this system.


Below is an implementation example of this method using our dataset:

In [5]:
A = np.array([x1[0]*P1[2,:3] - P1[0,:3],
              x1[1]*P1[2,:3] - P1[1,:3],
              x2[0]*P2[2,:3] - P2[0,:3],
              x2[1]*P2[2,:3] - P2[1,:3]])

b = np.array([P1[0,3] - x1[0]*P1[2,3],
              P1[1,3] - x1[1]*P1[2,3],
              P2[0,3] - x2[0]*P2[2,3],
              P2[1,3] - x2[1]*P2[2,3]])

# Solve the sistem Ax = b by least squares
X3 = np.linalg.pinv(A) @ b
print('Method 3 (inhomogeneous solution):\n{}'.format(X3))

Method 3 (inhomogeneous solution):
[ 54.13825235 -73.74546819 842.70530565]


We can see that the results are not equal to those obtained with the DLT method, but they are very close.

## Optimal triangulation method:

In case that we have a noisy correspondences of points $\mathbf{x}_1 \leftrightarrow \mathbf{x}_2$ which do not satisfy the epipolar constraint $\mathbf{x}_2^\mathsf{T} \mathbf{Fx}_1 = 0$. We can estimate a matches $\mathbf{\hat{x}}_1 \leftrightarrow \mathbf{\hat{x}}_2$ lying close to measured points $\mathbf{x}_1 \leftrightarrow \mathbf{x}_2$ and satisfting the epipolar constraint $\mathbf{\hat{x}}_2^\mathsf{T} \mathbf{F\hat{x}}_1 = 0$. In other words, we can estimate a points $\mathbf{\hat{x}}_1 \leftrightarrow \mathbf{\hat{x}}_2$ that minimize the geometric error cost function

\begin{equation}
\label{eq:cost}
C(\mathbf{\hat{x}}_1, \mathbf{\hat{x}}_2) = \left\lVert \mathbf{x}_1 - \mathbf{\hat{x}}_1 \right\rVert^2 + 
\left\lVert \mathbf{x}_2 - \mathbf{\hat{x}}_2 \right\rVert^2, \;\; \text{subject to} \;\; \mathbf{\hat{x}}_2^\mathsf{T} \mathbf{F\hat{x}}_1 = 0 \enspace,
\end{equation}

where $\left\lVert.\right\rVert$ is the $\ell_2$ norm.

This minimization problem is equivalent to minimize the reprojection error for a point $\mathbf{\hat{X}}$ which is mapped to $\mathbf{\hat{x}}_1$ and $\mathbf{\hat{x}}_2$ by projection matrices consistent with the fundamental matrix $\mathbf{F}$. The best solution to Eq.\ref{eq:cost} can be estimated by a non-iterative method, by solving a six-degree function. The strategy to optimal triangulation method is described in Algorithm 12.1 (pag. 318) from Multiple view geometry in computer vision - Hartley and Zisserman.

After finding the optial correspondences $\mathbf{\hat{x}}_1 \leftrightarrow \mathbf{\hat{x}}_2$, we can estimate the 3D position of the point using the homogenoeus method (DLT) described before.

We can correct the corresponding points with the OpenCV function `correctMatches` (documentation [here](https://docs.opencv.org/4.0.1/d9/d0c/group__calib3d.html#gaf32c99d17908e175ac71e7a08fad587b)). For that, we need the fundamental matrix which we do not know. Based on the known projection matrices $\mathbf{P}_1$ and $\mathbf{P}_2$, we can estimate the fundamental matrix with the expression

\begin{equation}
F_{ji} = (-1)^{i+j} \det \begin{bmatrix} \sim \!\! \mathbf{P}_1^i \\ \sim \!\! \mathbf{P}_2^j \end{bmatrix} \enspace,
\end{equation}

where $\sim \!\! \mathbf{P}^i$ denote the matrix $\mathbf{P}$ without the $i$-th row.

In [6]:
F = np.zeros([3,3])
for i in range(3):
    for j in range(3):
        F[j,i] = (-1)**(i+j)*np.linalg.det(np.r_[np.delete(P1,i,0), np.delete(P2,j,0)])
        
F = F/np.linalg.norm(F) # Scale normalization
print('Estimated fundamental matrix:\n{}'.format(F))

Estimated fundamental matrix:
[[-8.69272828e-09  8.42611092e-07 -3.93047042e-04]
 [ 6.66640455e-07  6.52244222e-08 -1.05988760e-02]
 [-4.24884400e-04  9.09605156e-03  9.99902291e-01]]


Then, an implementation of the optimal triangulation method:

In [7]:
# Enforce the epipolar constraint (correct matches)
x1h, x2h = cv2.correctMatches(F, x1.reshape(1,-1,2), x2.reshape(1,-1,2))

# Triangulate with homogeneous method
X4 = cv2.triangulatePoints(P1, P2, x1h, x2h)
X4 = (X4[:3]/X4[-1]).flatten()
print('Method 4 (Optimal triangulation):\n{}'.format(X4))

Method 4 (Optimal triangulation):
[ 54.13824938 -73.74544429 842.70532369]


We can evaluate the epipolar constrain with the measured and corrected matches to see the efect of the function `correctMatches`

In [8]:
# Evaluating the epipolar constraint before correct matches
print('Epipolar constraint before correct matches: {}'.format(np.r_[x2,1] @ F @ np.r_[x1,1]))

# Evaluate the epipolar constraint with corrected matches
print('Epipolar constraint with corrected matches: {}'.format(np.r_[x2h[0,0,:],1] @ F @ np.r_[x1h[0,0,:],1]))

Epipolar constraint before correct matches: 0.0016161348640753026
Epipolar constraint with corrected matches: 4.440892098500626e-16


# Triangulation in a fringe projection profilometry system

In case that we project horizontal and vertical fringes onto a scene, we can recover the $x_2$ and $y_2$ coordinates in the projector corresponding to a point $(x_1,\,y_1)$ in the camera, based on the recovered horizontal and vertical phase maps. If we know both the $x_2$ and $y_2$ coordinates in the projector we can use all the aforementioned strategies to perform a 3D reconstruction between the camera and the projector. But in case that, for example, we only project vertical fringes, we will only know the $x_2$ coordinates in the projector corresponding to $x_1$ and $y_1$ coordinates in the camera. For this case, we 
cannot use the triangulation methods already discussed.

As we only know the $x_2$ in the projector and not the $y_2$, we cannot address the triangulation in a homogeneous approach. Instead we have to address it in a inhomogeneous approach. This is due to, as discussed above, the homogeneous method has the same number of equations than unknowns (four equations and four unknowns), and the inhomogeneous method has more equations than unknowns (four equations and three unknowns) and in this case we can remove an equation of the system. This equations is the last row from Eq.\ref{eq:inh} which has the unknown $y_2$ coordinate of the projector. Removing this equation we have a system of three equantions and three unknowns.

In other words, we can write the projection equations $s_1 \mathbf{x}_1 = \mathbf{P}_1 \mathbf{X}$ and $s_2 \mathbf{x}_2 = \mathbf{P}_2 \mathbf{X}$ in an inhomogeneous approach as Eq.\ref{eq:pinhole1} and Eq.\ref{eq:pinhole2}, and get the following equations stated in the inhomogeneous section


\begin{align}
s_1 x_1 & = p^{11}_1 X + p^{12}_1 Y + p^{13}_1 Z + p^{14}_1 \enspace, \\
s_1 y_1 & = p^{21}_1 X + p^{22}_1 Y + p^{23}_1 Z + p^{24}_1 \enspace, \\
s_1     & = p^{31}_1 X + p^{32}_1 Y + p^{33}_1 Z + p^{34}_1 \enspace, \\
s_2 x_2 & = p^{11}_2 X + p^{12}_2 Y + p^{13}_2 Z + p^{14}_2 \enspace, \\
s_2 y_2 & = p^{21}_2 X + p^{22}_2 Y + p^{23}_2 Z + p^{24}_2 \enspace, \\
s_2     & = p^{31}_2 X + p^{32}_2 Y + p^{33}_2 Z + p^{34}_2 \enspace,
\end{align}

which can be written as

\begin{align}
\label{eq:inh1}
(p^{31}_1x_1-p^{11}_1)X + (p^{32}_1x_1-p^{12}_1)Y + (p^{33}_1x_1-p^{13}_1)Z & = p^{14}_1-p^{34}_1x_1 \enspace, \\
(p^{31}_1y_1-p^{21}_1)X + (p^{32}_1y_1-p^{22}_1)Y + (p^{33}_1y_1-p^{23}_1)Z & = p^{24}_1-p^{34}_1y_1 \enspace, \\
(p^{31}_2x_2-p^{11}_2)X + (p^{32}_2x_2-p^{12}_2)Y + (p^{33}_2x_2-p^{13}_2)Z & = p^{14}_2-p^{34}_2x_2 \enspace, \\
\label{eq:inh2}
(p^{31}_2y_2-p^{21}_2)X + (p^{32}_2y_2-p^{22}_2)Y + (p^{33}_2y_2-p^{23}_2)Z & = p^{24}_2-p^{34}_2y_2 \enspace. \\
\end{align}

In the system from Eq.\ref{eq:inh1} to Eq.\ref{eq:inh2}, we have four equations and five unknowns ($X$, $Y$, $Z$, $x_2$, and $y_2$). But based on the recovered absolute phase map $\Phi$ with the camera, we can estimate $x_2$ and reduce the number of unknowns to four, using the expression

\begin{equation}
x_2 = \Phi \dfrac{p}{2\pi} \enspace,
\end{equation}

where $p$ is the pith of the projected fringes. In this way we have four equations and four unknowns and the system can be solved. As we need to solve for ($X$, $Y$, $Z$), we can "ignore" the equation Eq.\ref{eq:inh2} that only can be used to estimate $y_2$ and is the only one that has the unknown $y_2$ coordinate. In this manner, we convert the system in a $3\times 3$ system of three equantions

\begin{align}
(p^{31}_1x_1-p^{11}_1)X + (p^{32}_1x_1-p^{12}_1)Y + (p^{33}_1x_1-p^{13}_1)Z & = p^{14}_1-p^{34}_1x_1 \enspace, \\
(p^{31}_1y_1-p^{21}_1)X + (p^{32}_1y_1-p^{22}_1)Y + (p^{33}_1y_1-p^{23}_1)Z & = p^{24}_1-p^{34}_1y_1 \enspace, \\
(p^{31}_2x_2-p^{11}_2)X + (p^{32}_2x_2-p^{12}_2)Y + (p^{33}_2x_2-p^{13}_2)Z & = p^{14}_2-p^{34}_2x_2 \enspace,
\end{align}

and three unknowns $X$, $Y$, and $Z$, which we can express matrically as

\begin{equation}
\label{eq:inh_unique_sol}
\begin{bmatrix}
p^{31}_1x_1-p^{11}_1 & p^{32}_1x_1-p^{12}_1 & p^{33}_1x_1-p^{13}_1\\
p^{31}_1y_1-p^{21}_1 & p^{32}_1y_1-p^{22}_1 & p^{33}_1y_1-p^{23}_1\\
p^{31}_2x_2-p^{11}_2 & p^{32}_2x_2-p^{12}_2 & p^{33}_2x_2-p^{13}_2
\end{bmatrix} 
\begin{bmatrix}
X\\Y\\Z
\end{bmatrix} = 
\begin{bmatrix}
p^{14}_1-p^{34}_1x_1\\
p^{24}_1-p^{34}_1y_1\\
p^{14}_2-p^{34}_2x_2
\end{bmatrix} \enspace,
\end{equation}

that is a system with unique solution unlike to the inhomogeneous method that has a least squares solution. And as you can see, the system is the equivalent to remove the last row from the inhomogeneous method given in Eq.\ref{eq:inh}.

An implementation using our dataset:

In [9]:
A = np.array([x1[0]*P1[2,:3] - P1[0,:3],
              x1[1]*P1[2,:3] - P1[1,:3],
              x2[0]*P2[2,:3] - P2[0,:3]])

b = np.array([P1[0,3] - x1[0]*P1[2,3],
              P1[1,3] - x1[1]*P1[2,3],
              P2[0,3] - x2[0]*P2[2,3]])

# Solve the sistem Ax = b with unique solution
X = np.linalg.inv(A) @ b
print('Inhomogenous solution with unique solution:\n{}'.format(X))

Inhomogenous solution with unique solution:
[ 54.13774066 -73.71957585 842.70589424]


The problem of not knowing both the $x_2$ and $y_2$ coordinates in the projector is that we cannot compensate for lens distortions the coordinates in the second view. We can only undistort the points in the camera, and it makes less accurate the reconstruction.

## A faster approach

We can also address this problem in a different way, which is efficient to code implementation if we align the world coordinate frame with the (camera) first view coordinate frame. In this way we write the projection matrices as:

\begin{equation}
\mathbf{P}_1 = \mathbf{K}_1
\begin{bmatrix} 
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0
\end{bmatrix} = \mathbf{K}_1 [\mathbf{I} \,\,|\,\, \mathbf{0}] \enspace,
\end{equation}

\begin{equation}
\mathbf{P}_2 = \mathbf{K}_2
\begin{bmatrix} 
r_{11} & r_{12} & r_{12} & t_x \\
r_{21} & r_{22} & r_{23} & t_y \\
r_{31} & r_{32} & r_{33} & t_z
\end{bmatrix} = \mathbf{K}_2 [\mathbf{R} \,\,|\,\, \mathbf{t}] \enspace,
\end{equation}

where $\mathbf{R}$ and $\mathbf{t}$ represent the position and orientation of camera/world frame (first view), realtive to the projector frame (second view). Then, the projection equations

\begin{equation}
s_1 \mathbf{x}_1 = \mathbf{K}_1 [\mathbf{I} \,\,|\,\, \mathbf{0}] \mathbf{X} \longrightarrow
s_1 \mathbf{K}_1^{-1} \mathbf{x}_1 = [\mathbf{I} \,\,|\,\, \mathbf{0}] \mathbf{X} \enspace,
\end{equation}

\begin{equation}
s_2 \mathbf{x}_2 = \mathbf{K}_2 [\mathbf{R} \,\,|\,\, \mathbf{t}] \mathbf{X} \longrightarrow
s_2 \mathbf{K}_2^{-1} \mathbf{x}_2 = [\mathbf{R} \,\,|\,\, \mathbf{t}] \mathbf{X} \enspace,
\end{equation}

where $\mathbf{\bar{x}}_1 = \mathbf{K}_1^{-1} \mathbf{x}_1$ and $\mathbf{\bar{x}}_2 = \mathbf{K}_2^{-1} \mathbf{x}_2$, are the normalized image coordinates in the camera and the projector image plane. With this normalized coordinates the above equations:

\begin{align}
\label{eq:normalizedproj1}
s_1 \begin{bmatrix} \bar{x}_1\\ \bar{y}_1\\1 \end{bmatrix} &  = 
\begin{bmatrix} 
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0
\end{bmatrix} \begin{bmatrix} X\\Y\\Z\\1 \end{bmatrix} \enspace, \\
\label{eq:normalizedproj2}
s_2 \begin{bmatrix} \bar{x}_2\\ \bar{y}_2\\1 \end{bmatrix} &  = 
\begin{bmatrix} 
r_{11} & r_{12} & r_{12} & t_x \\
r_{21} & r_{22} & r_{23} & t_y \\
r_{31} & r_{32} & r_{33} & t_z
\end{bmatrix} \begin{bmatrix} X\\Y\\Z\\1 \end{bmatrix} \enspace.
\end{align}

From Eq.\ref{eq:normalizedproj1} we have the following three equations

\begin{align}
s_1 \bar{x}_1 & = X \enspace,\\
s_1 \bar{y}_1 & = Y \enspace,\\
s_1     & = Z \enspace,
\end{align}

or equivalently

\begin{align}
\label{eq:normprojX1}
X = Z \bar{x}_1 \enspace,\\
\label{eq:normprojY1}
Y = Z \bar{y}_1 \enspace.
\end{align}



Now, from Eq.\ref{eq:normalizedproj2} we get three equations:

\begin{align}
\label{eq:normprojX2}
s_2 \bar{x}_2 & = r_{11} X + r_{12} Y + r_{13} Z + t_x \enspace,\\
\label{eq:normprojY2}
s_2 \bar{y}_2 & = r_{21} X + r_{22} Y + r_{23} Z + t_y \enspace,\\
\label{eq:normprojZ2}
s_2     & = r_{31} X + r_{32} Y + r_{33} Z + t_z \enspace.
\end{align}

Replacing Eq.\ref{eq:normprojZ2} into Eq.\ref{eq:normprojX2} and Eq.\ref{eq:normprojY2} and grouping the unknowns $X$, $Y$ and $Z$:
\begin{align}
\label{eq:normprojx22}
(r_{31}\bar{x}_2-r_{11})X + (r_{32}\bar{x}_2-r_{12})Y + (r_{33}\bar{x}_2-r_{13})Z & = t_x-t_z \bar{x}_2 \enspace,\\
\label{eq:normprojy22}
(r_{31}\bar{y}_2-r_{21})X + (r_{32}\bar{y}_2-r_{22})Y + (r_{33}\bar{y}_2-r_{23})Z & = t_y-t_z \bar{y}_2 \enspace.
\end{align}

Here we have four equations (Eq.\ref{eq:normprojX1}, Eq.\ref{eq:normprojY1}, Eq.\ref{eq:normprojx22}, and Eq.\ref{eq:normprojy22}) and four unknowns ($X$, $Y$, $Z$, and $\bar{y}_2$). As discussed in the previos approach, as we need to solve for ($X$, $Y$, $Z$) we can "ignore" an equation. Due to Eq.\ref{eq:normprojy22} has the unknown $\bar{y}_2$, we can use it to calculate the $\bar{y}_2$ coordinate in the second view. Then, replacing Eq.\ref{eq:normprojX1} and Eq.\ref{eq:normprojY1} into Eq.\ref{eq:normprojx22}, we have:

\begin{equation}
(r_{31}\bar{x}_2-r_{11})Z \bar{x}_1 + (r_{32}\bar{x}_2-r_{12})Z \bar{y}_1 + (r_{33}\bar{x}_2-r_{13})Z = t_x-t_z \bar{x}_2
\enspace,
\end{equation}

where

\begin{equation}
Z = \dfrac{t_x-t_z \bar{x}_2}{(r_{31}\bar{x}_2-r_{11}) \bar{x}_1 + (r_{32}\bar{x}_2-r_{12}) \bar{y}_1 + (r_{33}\bar{x}_2-r_{13})}  \enspace.
\end{equation}

Once we estimate the $Z$ coordinate, with Eq.\ref{eq:normprojX1} and Eq.\ref{eq:normprojY1} we can estimate the $X$ and $Y$ coordinates of the 3D point, respectively.


An implementation with our dataset:

In [10]:
# Normalizing the image coordinates
x1b, y1b, _ = np.linalg.inv(K1) @ np.r_[x1, 1]
x2b, *_ = np.linalg.inv(K2) @ np.r_[x2, 1] # We don't need y coor

# Estimate z coordinate
Z = (t[0]-t[2]*x2b)/((R[2,0]*x2b-R[0,0])*x1b + (R[2,1]*x2b-R[0,1])*y1b + R[2,2]*x2b-R[0,2])

# 3D point
X = np.array([x1b*Z, y1b*Z, Z])
print('Inhomogenous solution with unique solution (second approach):\n{}'.format(X))

Inhomogenous solution with unique solution (second approach):
[ 54.13774592 -73.719583   842.70597599]


The implementation of this approach is more computationally efficient than finding iteratively for each point the solution with the inverse matrix in Eq.\ref{eq:inh_unique_sol}. In this way, we can speed up the reconstruction process, especially where the number of points is large.


## Estimating the $y$ coordinate of the projector


If we only project phase in the $x$ direction, we only have the $x_2$ coorinates in the projector. With the aim to use the homoegenous (Eq.\ref{eq:DLT1}), inhomogeneous (Eq.\ref{eq:inh}), and the optimal (Eq.\ref{eq:cost}) triangulation approaches, we need to estimate the $y_2$ coordinates in the projector, due to this methods need both the $x$ and $y$ coordinates in the first and second view. In high speed applications, where we cannot project phase in $y$ direction, we can estimate geomtrically the $y_2$ coordinates in the second view to use the aforementioned approaches. Based on the fundamental matrix $\mathbf{F}$ between the two views, we can estimate the epipolar line $\mathbf{l}_2 = [a,b,c]^\mathsf{T}$ in the projector DMD, and with this line estimate the $y_2$ coordinate in the projector.

With the expression

\begin{equation}
\mathbf{l}_2 = \mathbf{Fx}_1 \enspace,
\end{equation}

we estimate the epipolar line in the second view. Then, from the line $ax_2+by_2+c=0$ we can estimate the $y_2$ as

\begin{equation}
y_2 = -\dfrac{ax_2+c}{b} \enspace,
\end{equation}

and with this estimated coordinate use the different approaches that need both coordinate to triangulate.

As an example with our dataset, let's estimate the $y_2$ coordinate in the second view and, with the DLT method, the 3D coordinates of the point.

In [11]:
# Estimating the epipolar line
l2 = F @ np.r_[x1, 1]

# Estimate y coordinate in the second view
y2 = -(l2[0]*x1[0]+l2[2])/l2[1]
print('Estimated y coordinate of the projector: {}'.format(y2))

# Triangulate with homogeneous method
X = cv2.triangulatePoints(P1, P2, x1, np.r_[x2[0], y2])
X = (X[:3]/X[-1]).flatten()
print('\n3D point with estimated y coordinate in the projector:\n{}'.format(X))

Estimated y coordinate of the projector: 359.4018875545487

3D point with estimated y coordinate in the projector:
[ 54.14595203 -74.13219518 842.69678686]


This can be useful to undistort the points in the projector DMD and perform a more accurate 3D reconstruction, as to undistort the points we need both coordinates.